In [ ]:

### create water network model
begin
    net_path = "/home/bradw/workspace/networks/data/"
    # net_name = "bwfl_2022_05_hw"
    net_name = "modena"

    if net_name == "bwfl_2022_05_hw"
        net_dir = "bwfl_2022_05/hw"
        inp_file = net_path * net_dir * "/" * net_name * ".inp"
    else
        inp_file = net_path * net_name * "/" * net_name * ".inp"
    end

    wn = wntr.network.WaterNetworkModel(inp_file)

    # store original network element names
    reservoir_names = wn.reservoir_name_list
    junction_names = wn.junction_name_list
    link_names = wn.link_name_list
    pipe_names = wn.pipe_name_list
    valve_names = wn.valve_name_list

end

### simulate network hydraulics and convert to Julia DataFrame
begin
    num_days = 1
    wn.options.time.duration = num_days*24*3600
    nt = Int(wn.options.time.duration ./ wn.options.time.hydraulic_timestep)
    wn.options.time.rule_timestep = wn.options.time.hydraulic_timestep
    wn.convert_controls_to_rules(priority=3) # convert controls to rules

    wn.options.quality.parameter = "CHEMICAL"
    wn.options.reaction.bulk_coeff = -2.5e-05 # units = 1/second
    wn.options.reaction.wall_coeff = 0.0

    source_node_1 = reservoir_names[1]
    d_1 = Truncated(Normal(0.5, 0.0), 0, Inf)
    source_pattern_1 = rand(d_1, Int(nt/num_days))
    wn.add_pattern("source_pattern_1", source_pattern_1)
    wn.add_source("source_1", source_node_1, "CONCEN", 1, "source_pattern_1")

    source_node_2 = reservoir_names[2]
    d_2 = Truncated(Normal(0.65, 0.0), 0, Inf)
    source_pattern_2 = rand(d_2, Int(nt/num_days))
    wn.add_pattern("source_pattern_2", source_pattern_2)
    wn.add_source("source_2", source_node_2, "CONCEN", 1, "source_pattern_2")

    # wn.options.quality.parameter = "AGE"
    # wn.options.quality.parameter = "TRACE"
    # wn.options.quality.trace_node = "node_2746"

    sim = wntr.sim.EpanetSimulator(wn)
    results = sim.run_sim()

    # head values at nodes
    df_head = DataFrame()
    df_head[!, "timestamp"] = results.node["head"].index.values ./ 3600
    for col ∈ results.node["head"].columns
        df_head[!, col] = getproperty(results.node["head"], col).values
    end

    # quality values at nodes
    df_qual = DataFrame()
    df_qual[!, "timestamp"] = results.node["quality"].index.values ./ 3600
    for col ∈ results.node["quality"].columns
        df_qual[!, col] = getproperty(results.node["quality"], col).values #./ 3600 # covnert to water age to hours
    end

    # flow values at links
    df_flow = DataFrame()
    df_flow[!, "timestamp"] = results.node["head"].index.values ./ 3600
    for col ∈ results.link["flowrate"].columns
        df_flow[!, col] = getproperty(results.link["flowrate"], col).values .* 1000
    end

end